In [10]:
import pandas as pd
import numpy as np

def sigmoid(z):
  return 1/(1+np.exp(-z))

def initialize(n_features):
  weights=np.zeros(n_features)
  bias=0;
  return weights,bias

def propogate(X, y, weights, bias):
    m = X.shape[0]
    A = sigmoid(np.dot(X, weights) + bias)
    epsilon = 1e-15
    A = np.clip(A, epsilon, 1 - epsilon)

    cost = (-1 / m) * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))
    dw = (1 / m) * np.dot(X.T, (A - y))
    db = (1 / m) * np.sum(A - y)

    return dw, db, cost

def train(X,y,lr=0.1,n_iterations=1000):
  weights,bias=initialize(X.shape[1])

  for i in range(0,1000):
    dw,db,cost=propogate(X,y,weights,bias)
    weights=weights-lr*dw
    bias=bias-lr*db

  return weights,bias


def predict(X,weights,bias):
  A=sigmoid(bias+np.dot(X,weights))

  return (A>=0.5).astype(int)

def accuracy(y_true,y_pred):

  return np.mean(y_true==y_pred)


from sklearn.datasets import make_classification

X1, y = make_classification(n_samples=500, n_features=2, n_informative=2, n_redundant=0, n_classes=2, random_state=42)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X1)


weights,bias=train(X,y)
y_pred=predict(X,weights,bias)
acc=accuracy(y,y_pred)
print(f"Training Accuracy= {acc*100:.2f}%")

Training Accuracy= 89.00%
